In [2]:
import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import math

np.random.seed(248745)

In [3]:
class Hotspot():
    def __init__(self, number, profits, area, uuv_obs_rate):
        self.number = number
        self.profits = profits
        self.area = area
        self.lam = uuv_obs_rate/area

##Table of expected profits


##Table of approximate path costs for now
 

In [134]:
##import all hotspot to hotspot travel time over all time intervals
with open('./../santa_monica_energy_costs.npy', 'rb') as f:
    transit_times = np.load(f)
    
# transit_times = np.random.randint(0, 1000, size=(7,7,1))

##build all the hotspot data
uuv_obs_rate = (5**2)*math.pi #arbitrarily selected observation rate
all_hotspots = [Hotspot(0, 0, 0.1, uuv_obs_rate), ##This is the init position
                Hotspot(1, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(2, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(3, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(4, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(5, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(6, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                ]

In [135]:
transit_times.shape

(6, 6, 144)

In [102]:
transit_times.shape

transit_dict = {}
for i in range(len(all_hotspots)):
    for j in range(len(all_hotspots)):
        transit_dict[(str(i),str(j))] = transit_times[i,j,0]

In [79]:
##Callback - use lazy constraints to eliminate subtours
##Stolen from the Gurobi TSP example

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

## Objective Function
$$\max \sum_{i=1}^{N-1} \sum_{j=2}^{N} (P_i x_{ij} - t_{ij} x_{ij}) $$

Where:

$P_i$ = profit at each hotspot, given by the equation $f(t_i) = 1 - \exp^{-\lambda t_i}$

A profit of 0 is associated with nodes 1 and N. Make this the starting point.

$\lambda_i$ = sensory learning rate, given by $\frac{obs_r^2 \pi}{A_i}$, where $obs_r^2 \pi$ = UUV sensing rate, and $A_i$ = area of hotspot

$x_{ij}$ = binary, 1 if a visit to hotspot $i$ is followed by a visit to hotspot $j$

$t_{ij}$ = Non-negative travel cost between hotspots $i$ and $j$

## Constraints

Ensures the route starts from node 1:

$$\sum_{j=2}^{N} x_{1j} = \sum_{i=1}^{N-1} x_{iN}= 1$$

Ensures the connectivity of the route and that each onde is visited at most once:

$$\sum_{i=1}^{N-1} x_{ik} = \sum_{j=2}^{N} x_{kj} \leq 1; \forall k = 2, \dots, (N-1)$$

Prevents subtours:

$$2 \leq u_i \leq N; \forall i = 2, ..., N$$

$$u_i - u_j + 1 \leq (N - 1)(1 - x_{ij}); \forall j = 2, ..., N$$

In [99]:
transit_dict

{('0', '0'): 750,
 ('0', '1'): 297,
 ('0', '2'): 756,
 ('0', '3'): 569,
 ('0', '4'): 492,
 ('0', '5'): 338,
 ('1', '0'): 127,
 ('1', '1'): 701,
 ('1', '2'): 920,
 ('1', '3'): 99,
 ('1', '4'): 768,
 ('1', '5'): 657,
 ('2', '0'): 816,
 ('2', '1'): 767,
 ('2', '2'): 960,
 ('2', '3'): 489,
 ('2', '4'): 583,
 ('2', '5'): 501,
 ('3', '0'): 183,
 ('3', '1'): 127,
 ('3', '2'): 286,
 ('3', '3'): 644,
 ('3', '4'): 175,
 ('3', '5'): 424,
 ('4', '0'): 884,
 ('4', '1'): 200,
 ('4', '2'): 582,
 ('4', '3'): 355,
 ('4', '4'): 232,
 ('4', '5'): 906,
 ('5', '0'): 158,
 ('5', '1'): 17,
 ('5', '2'): 963,
 ('5', '3'): 894,
 ('5', '4'): 553,
 ('5', '5'): 612}

In [110]:
hotspot_idx = [k.number for k in all_hotspots]
hotspot_profits = [k.profits for k in all_hotspots]


model = gp.Model("basic_op")

##x_ij boolean is whehter the hotspot i to j (bool) has been taken
x = model.addVars(transit_dict.keys(), vtype=GRB.BINARY, name="hotspot_visited")

##t_i is a float value that explains how much time is spent at each node
t = model.addVars(hotspot_idx, lb=0, ub=200, vtype=GRB.CONTINUOUS, name="time_at_node")

##t_ij boolean is whether the path from i to j was taken
# t = model.addVars(transit_dict.keys(), vtype=GRB.BINARY, name="path_taken")

##Constraints
##Route starts from node 0
model.addConstr(gp.quicksum( [x[str(0), str(j)] for j in range(1, len(hotspot_idx))]) == 1) 

model.addConstr(gp.quicksum( [x[str(j), str(0)] for j in range(1, len(hotspot_idx))]) == 1)


##Route connectivity and nodes are visited only once
for k in range(1, len(all_hotspots)-1):
    one = gp.quicksum( [x[str(i), str(k)] for i in range(0, len(all_hotspots)-1)] )
    two = gp.quicksum( [x[str(k), str(j)] for j in range(1, len(all_hotspots))]) 
    model.addConstr(one == two)
    model.addConstr(one <= 1)

##subtour constraints
##See callback function above


##Objective function
y = model.addVars(hotspot_idx, vtype=GRB.CONTINUOUS, name="exp_value")
for i in hotspot_idx:
    ex = model.addGenConstrPWL(t[i], y[i], np.arange(0, 200), np.exp(-hotspot_profits[i] * np.arange(0, 200))
                            )

all_profits = []
all_paths = []
for i in range(1, len(hotspot_idx)-1):
    for j in range(2, len(hotspot_idx)):
#         time_spent = (1 - y[i])  * x[str(i),str(j)]
        time_spent = 1 + t[i]*50
        all_profits.append(time_spent)
        
        paths_cost = transit_dict[str(i),str(j)] * x[str(i), str(j)]
        all_paths.append(paths_cost)
        
model.setObjective(gp.quicksum(all_profits + all_paths)
                  )

In [111]:
model._vars = [x,y,t]
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 12 rows, 63 columns and 92 nonzeros
Model fingerprint: 0xb807bd03
Model has 7 general constraints
Variable types: 14 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 1e+00]
Presolve added 0 rows and 138 columns
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 201 columns, 400 nonzeros
Presolved model has 1 SOS constraint(s)
Variable types: 201 continuous, 0 integer (0 binary)

Root relaxation: objective 2.500000e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      25.0000000   25.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 16 

In [91]:
model._vars = [x, y, t]
model.Params.lazyConstraints = 1
model.optimize(subtourelim)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 11 rows, 50 columns and 76 nonzeros
Model fingerprint: 0x6e4204ba
Model has 25 quadratic objective terms
Model has 7 general constraints
Variable types: 14 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+08]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 1e+00]
Presolve added 19 rows and 1398 columns
Presolve time: 0.00s
Presolved: 55 rows, 1473 columns, 2963 nonzeros
Presolved model has 7 SOS constraint(s)
Variable types: 1437 continuous, 36 integer (36 binary)


AttributeError: 'tupledict' object has no attribute 'indexed_data'

Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 42, time in user-callback 0.01 sec


Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "<ipython-input-79-b9616c218ab7>", line 7, in subtourelim
  File "model.pxi", line 5935, in gurobipy.Model.cbGetSolution
  File "model.pxi", line 5894, in gurobipy.Model._map_coldata_to_vars
AttributeError: 'tupledict' object has no attribute 'indexed_data'


In [112]:
model.getVars()

[<gurobi.Var hotspot_visited[0,0] (value 0.0)>,
 <gurobi.Var hotspot_visited[0,1] (value -0.0)>,
 <gurobi.Var hotspot_visited[0,2] (value -0.0)>,
 <gurobi.Var hotspot_visited[0,3] (value -0.0)>,
 <gurobi.Var hotspot_visited[0,4] (value -0.0)>,
 <gurobi.Var hotspot_visited[0,5] (value -0.0)>,
 <gurobi.Var hotspot_visited[0,6] (value 1.0)>,
 <gurobi.Var hotspot_visited[1,0] (value 1.0)>,
 <gurobi.Var hotspot_visited[1,1] (value 0.0)>,
 <gurobi.Var hotspot_visited[1,2] (value -0.0)>,
 <gurobi.Var hotspot_visited[1,3] (value -0.0)>,
 <gurobi.Var hotspot_visited[1,4] (value -0.0)>,
 <gurobi.Var hotspot_visited[1,5] (value -0.0)>,
 <gurobi.Var hotspot_visited[1,6] (value -0.0)>,
 <gurobi.Var hotspot_visited[2,0] (value 0.0)>,
 <gurobi.Var hotspot_visited[2,1] (value -0.0)>,
 <gurobi.Var hotspot_visited[2,2] (value 0.0)>,
 <gurobi.Var hotspot_visited[2,3] (value -0.0)>,
 <gurobi.Var hotspot_visited[2,4] (value -0.0)>,
 <gurobi.Var hotspot_visited[2,5] (value -0.0)>,
 <gurobi.Var hotspot_visit

In [85]:
model.getAttr('x', x)

{('1', '1'): 1.0,
 ('1', '2'): 0.0,
 ('1', '3'): 0.0,
 ('1', '4'): 0.0,
 ('1', '5'): 0.0,
 ('1', '6'): 0.0,
 ('2', '1'): 0.0,
 ('2', '2'): 0.0,
 ('2', '3'): 0.0,
 ('2', '4'): 0.0,
 ('2', '5'): 0.0,
 ('2', '6'): 0.0,
 ('3', '1'): 0.0,
 ('3', '2'): 0.0,
 ('3', '3'): 0.0,
 ('3', '4'): 0.0,
 ('3', '5'): 0.0,
 ('3', '6'): 0.0,
 ('4', '1'): 0.0,
 ('4', '2'): 0.0,
 ('4', '3'): 0.0,
 ('4', '4'): 0.0,
 ('4', '5'): 0.0,
 ('4', '6'): 0.0,
 ('5', '1'): 0.0,
 ('5', '2'): 0.0,
 ('5', '3'): 0.0,
 ('5', '4'): 0.0,
 ('5', '5'): 0.0,
 ('5', '6'): 0.0,
 ('6', '1'): 0.0,
 ('6', '2'): 0.0,
 ('6', '3'): 0.0,
 ('6', '4'): 0.0,
 ('6', '5'): 0.0,
 ('6', '6'): 0.0}

In [86]:
model.getAttr("t", t)

AttributeError: 'gurobipy.Model' object has no attribute 't'

In [129]:
import random
import gurobipy as grb
import math

n = 4
# Distance = 50000000
Distance = 100

def distance(points, i, j):
  dx = points[i][0] - points[j][0]
  dy = points[i][1] - points[j][1]
  return math.sqrt(dx*dx + dy*dy)

random.seed(1)
points = []
for i in range(n):
  points.append((random.randint(0,100),random.randint(0,100)))


opt_model = grb.Model(name="MILP Model")

# <= Variables
x_vars = {}
for i in range(n):
   for j in range(n):
     x_vars[i,j] = opt_model.addVar(vtype=grb.GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))
u={}
for i in range(1,n):
    u[i]=opt_model.addVar(vtype=grb.GRB.INTEGER,
                          name='e'+str(i))

# <= Constraint (Mandatory Edges and excluding vertexes) Eq(1)


opt_model.addConstr((grb.quicksum(x_vars[1,j] for j in range(1,n)))  == 1)
opt_model.addConstr((grb.quicksum(x_vars[i,n-1] for i in range(n-1)))  == 1)
# opt_model.addConstr((grb.quicksum(x_vars[i,i] for i in range(n-1)))  == 0)
# <= Constraint (Distance) Eq(3)

for i in range(n-1):
  opt_model.addConstr(grb.quicksum(x_vars[i,j]*distance(points, i, j) for j in range(1,n)) <= Distance)



# <= Constraint (Equality & Single edge in and out) Eq(2)

for k in range(1, n-1):
#   opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
#                       == grb.quicksum(x_vars[k,j] for j in range(1, n)) <=1)

    one = grb.quicksum(x_vars[i,k] for i in range(n-1))
    two = grb.quicksum(x_vars[k,j] for j in range(1, n))
    opt_model.addConstr(one == two)
    opt_model.addConstr(one <= 1)

# <= Constraint (Subtour elimination) Eq(4) Eq(5)

for i in range(1,n):
    opt_model.addConstr(2 <= u[i])
    opt_model.addConstr(u[i] <= n)
#   opt_model.addConstr(2 <= u[i] <= n)

for i in range(1,n):
    for j in range(1,n):
        opt_model.addConstr((u[i] - u[j] +1 <= (n-1)*(1-x_vars[j,i])))

# <= objective (maximize) Eq(1)

objective = grb.quicksum(x_vars[i,j]
                         for i in range(1, n-1)
                         for j in range(1, n))

opt_model.ModelSense = grb.GRB.MAXIMIZE
opt_model.setObjective(objective)
opt_model.optimize()


opt_model.update()
# opt_model.computeIIS()
# opt_model.write("model.ilp")
solution = opt_model.getAttr('x', x_vars )

print (solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 24 rows, 19 columns and 54 nonzeros
Model fingerprint: 0x86e4422f
Variable types: 0 continuous, 19 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 2.0000000
Presolve removed 24 rows and 19 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
{(0, 0): 0.0, (0, 1): 0.0, (0, 2): 1.0, (0, 3): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 1.0, (2, 0): 0.0, (2, 1): 1.0, (2, 2): 0.0, (2, 3): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0}


In [130]:
select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)

In [131]:
select

<gurobi.tuplelist (3 tuples, 2 values each):
 ( 0 , 2 )
 ( 1 , 3 )
 ( 2 , 1 )
>

In [122]:
points

[(17, 72), (97, 8), (32, 15), (63, 97)]

In [128]:
total = distance(points, 0,1) + distance(points, 1,2) + distance(points, 2, 3)
print (total)

255.48995608942263


In [132]:
total = distance(points, 0,2) + distance(points, 1,3) + distance(points, 2, 1)
print (total)

219.58977574350132


In [ ]:
import random
import gurobipy as grb

n = 4
Distance = 50000000

def distance(points, i, j):
  dx = points[i][0] - points[j][0]
  dy = points[i][1] - points[j][1]
  return math.sqrt(dx*dx + dy*dy)

random.seed(1)
points = []
for i in range(n):
  points.append((random.randint(0,100),random.randint(0,100)))


opt_model = grb.Model(name="MILP Model")

# <= Variables
x_vars = {}
for i in range(n):
   for j in range(n):
     x_vars[i,j] = opt_model.addVar(vtype=grb.GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))
u={}
for i in range(1,n):
    u[i]=opt_model.addVar(vtype=grb.GRB.INTEGER,
                          name='e'+str(i))

# <= Constraint (Mandatory Edges and excluding vertexes) Eq(1)


opt_model.addConstr((grb.quicksum(x_vars[1,j] for j in range(1,n)))  == 1)
opt_model.addConstr((grb.quicksum(x_vars[i,n-1] for i in range(n-1)))  == 1)
# opt_model.addConstr((grb.quicksum(x_vars[i,i] for i in range(n-1)))  == 0)
# <= Constraint (Distance) Eq(3)

for i in range(n-1):
  opt_model.addConstr(grb.quicksum(x_vars[i,j]*distance(points, i, j) for j in range(1,n)) <= Distance)



# <= Constraint (Equality & Single edge in and out) Eq(2)

for k in range(1, n-1):
#   opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
#                       == grb.quicksum(x_vars[k,j] for j in range(1, n)) <=1)

    one = grb.quicksum(x_vars[i,k] for i in range(n-1))
    two = grb.quicksum(x_vars[k,j] for j in range(1, n))
    opt_model.addConstr(one == two)
    opt_model.addConstr(one <= 1)

# <= Constraint (Subtour elimination) Eq(4) Eq(5)

for i in range(1,n):
    opt_model.addConstr(2 <= u[i])
    opt_model.addConstr(u[i] <= n)
#   opt_model.addConstr(2 <= u[i] <= n)

for i in range(1,n):
    for j in range(1,n):
        opt_model.addConstr((u[i] - u[j] +1 <= (n-1)*(1-x_vars[j,i])))

# <= objective (maximize) Eq(1)

objective = grb.quicksum(x_vars[i,j]
                         for i in range(1, n-1)
                         for j in range(1, n))

opt_model.ModelSense = grb.GRB.MAXIMIZE
opt_model.setObjective(objective)
opt_model.optimize()


opt_model.update()
# opt_model.computeIIS()
# opt_model.write("model.ilp")
solution = opt_model.getAttr('x', x_vars )

print (solution)